In [ ]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from itertools import product
from tensorpac import Pac
from brainpipe.feat.utils._feat import _manageWindow

plt.style.use('seaborn-poster')

### Compute and Plot PAC values 
    Low frequency phases and large broad band gamma amplitude

In [ ]:
st = study('Olfacto')
Test = False
if Test == True:
    subjects = ['FERJ']
    conds = {'subj':('LowHigh',['low','high'])}
else:
    subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
    conds = {'subj':('LowHigh',['low','high'])}

### Compare different PAC methods and normalization

In [ ]:
for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_'+conds[cond][0]+'/')
    path2save = path.join(st.path,'feature/2_PAC_Encoding_EpiPerf_'+conds[cond][0]+'/')
    
    for c in conds[cond][1]:
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            data = np.load(loadname)
            data = data['x'][:,:2304,:].swapaxes(1,2)
            n_elec, n_trials, npts = data.shape
            sf, famp = 512., (13,120,2,1)
            fpha = (0.1,13,1,.2)
            print ('--> compute pac on : ', fi, 'data', data.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
            
            for elec in range(n_elec):
                data_elec = data[elec,:,:]
                print(data_elec.shape)
                p = Pac(fpha=fpha, famp=famp, filt='fir1', cycle=(3,6),dcomplex='hilbert')
                phases = p.filter(sf, data_elec, axis=1, ftype='phase')
                amplitudes = p.filter(sf, data_elec, axis=1, ftype='amplitude')
                print(phases.shape, amplitudes.shape)
                
                plt.figure(figsize=(22, 22))
                for i,k in enumerate(range(5)):
                    # Change the pac method :
                    p.idpac = (1, 1, k)
                    print('-> Normalization using ' + p.norm)
                    #print('-> PAC using ' + str(p))
                    # Compute only the PAC without filtering :
                    xpac = p.fit(phases, amplitudes, axis=2, nperm=100)
                    # Plot :
                    title = p.method + '\n' + p.surro + '\n' + p.norm
                    plt.subplot(4, 3, k+1)
                    p.comodulogram(xpac.mean(-1), title=title, cmap='Spectral_r')
                for i,j in enumerate(range(7,12,1)):
                    print(i,j)
                    plt.subplot(4,3,j)
                    p.idpac = (1, 1, i)
                    xpac_corr = p.filterfit(sf, data_elec, data_elec, axis=1)
                    title = p.method + '\n' + p.surro + '\n' + p.norm
                    p.comodulogram(xpac_corr.mean(-1), title=title, cmap='Spectral_r')
                plt.tight_layout()
                plt.show()
                0/0

### Compute and plot comodulograms

In [ ]:
for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_'+conds[cond][0]+'/')
    path2save = path.join(st.path,'feature/2_PAC_Encoding_EpiPerf_'+conds[cond][0]+'/Comodulograms/')
    
    for c in conds[cond][1]:
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            data = np.load(loadname)
            data,labels = data['x'][:,768:2816,:].swapaxes(1,2), data['Mai_RL']
            n_elec, n_trials, npts = data.shape
            sf, famp = 512., (13,120,3,1)
            fpha = (0.1,13,1,.2)
            print ('--> compute pac on : ', fi, 'data', data.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
            
            for elec in range(n_elec):
                data_elec = data[elec,:,:]
                #print(data_elec.shape)
                p = Pac(idpac=(1, 1, 4),fpha=fpha, famp=famp, filt='fir1', cycle=(3,6),dcomplex='hilbert')
                               
                plt.figure(figsize=(12,9))
                xpac_corr = p.filterfit(sf, data_elec, data_elec, axis=1)
                title = 'PAC - '+c.capitalize()+ ' Memory - '+ labels[elec]
                p.comodulogram(xpac_corr.mean(-1), title=title, plotas='imshow', 
                               cmap='plasma', vmin=0, interp=(.1,.1), vmax=0.7)
                filename = su+'_PAC_'+str(elec)+'_'+labels[elec]+'_'+c+'.png'
                plt.tight_layout()
                plt.savefig(path2save+filename)
                plt.clf()
                plt.close()